In [1]:
# import classes for python application
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

import re

import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# create spark session, configuration and context
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
conf = SparkConf().setMaster("local").setAppName("Assignment 1")
sc = SparkContext.getOrCreate(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/05 09:29:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#load text in an RDD instance and get text line count
linesrdd = sc.textFile('ulysses10.txt')
linesrdd.count()

32742

In [3]:
# check word count using a lambda function
result1 = linesrdd.filter(lambda x: bool(re.search('\\bafternoon\\b|\\bnight\\b|\\bmorning\\b', x))).count()
print('Result 1: ', result1)

Result 1:  326


In [4]:
# check word count using named function
def search_words(lines, reg='\\bafternoon\\b|\\bnight\\b|\\bmorning\\b'):
    for line in lines.split("\n"):
        result = re.findall(reg, line)
        count = len(result) > 0
        return count

result2 = linesrdd.filter(search_words).count()
print('Result 2: ', result2)

Result 2:  326


In [5]:
#split each line into words, create a tuple for each work and get the word count
counts = linesrdd.flatMap(lambda line: line.split(" ")) \
                            .map(lambda word: (word, 1)) \
                           .reduceByKey(lambda x, y: x + y)

In [6]:
# return records as a list
result = counts.collect()
result

[('The', 958),
 ('Project', 25),
 ('Gutenberg', 21),
 ('Etext', 6),
 ('of', 7566),
 ('Ulysses', 8),
 ('by', 1123),
 ('James', 30),
 ('Joyce', 5),
 ('(#4', 1),
 ('in', 4218),
 ('our', 234),
 ('series', 9),
 ('Joyce)', 1),
 ('', 10723),
 ('Copyright', 1),
 ('laws', 6),
 ('are', 411),
 ('changing', 3),
 ('all', 999),
 ('over', 348),
 ('the', 12660),
 ('world.', 40),
 ('Be', 22),
 ('sure', 66),
 ('to', 4542),
 ('check', 5),
 ('copyright', 9),
 ('for', 1751),
 ('your', 452),
 ('country', 36),
 ('before', 173),
 ('distributing', 2),
 ('this', 360),
 ('or', 905),
 ('any', 191),
 ('other', 268),
 ('file.', 4),
 ('We', 121),
 ('encourage', 4),
 ('you', 1346),
 ('keep', 52),
 ('file,', 1),
 ('exactly', 12),
 ('as', 1077),
 ('it', 1637),
 ('is,', 31),
 ('on', 1727),
 ('own', 128),
 ('disk,', 2),
 ('thereby', 7),
 ('keeping', 12),
 ('an', 541),
 ('electronic', 1),
 ('path', 10),
 ('open', 72),
 ('future', 16),
 ('readers.', 2),
 ('Please', 14),
 ('do', 291),
 ('not', 718),
 ('remove', 1),
 ('this.

In [7]:
# get the sum of the total number of words
total = 0
for(word, count) in result:
    total += count

#print total word count minus empty strings
print(total - 10723)

267832


In [8]:
#shutdown the spark context and spark
sc.stop()
spark.stop()